In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
with open('text/WoO.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
vocab_size = len(chars)

In [12]:
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [13]:
training_size = int(0.8*len(data))
train_data = data[:training_size]
test_data = data[training_size:]



In [14]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch("train")

print(x)
print(y)

tensor([[68, 53,  1, 63, 54,  1, 68, 56],
        [56, 53,  1, 54, 57, 70, 53,  1],
        [ 1, 43, 56, 53, 73,  1, 49, 66],
        [63, 70, 53, 66,  1, 68, 56, 53]], device='cuda:0')
tensor([[53,  1, 63, 54,  1, 68, 56, 53],
        [53,  1, 54, 57, 70, 53,  1, 64],
        [43, 56, 53, 73,  1, 49, 66, 53],
        [70, 53, 66,  1, 68, 56, 53,  1]], device='cuda:0')


In [17]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)

            index_next = torch.multinomial(probs, num_samples=1)

            index = torch.cat((index, index_next), dim=1)
        
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

tensor([[0]], device='cuda:0')
501


In [20]:
learning_rate = 3e-4
max_iters=10000
eval_iters=250

optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        print(f'loss: {loss}')

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb,yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

In [21]:
x = torch.tensor([-0.05], dtype=torch.float32)
y = nn.Re


wa, y tlyb;Neve SVan thengany simugr alige abrcepy,"T4z j4d tooke er
ope?"SKNmaug j4zeym to


foten, he cMQ38?
sowfund, f the an and Y&C!wabind yow y thesk!z?"N!" tsha8L6&;9&Cme the bee ab, f ate ime fthy scagy b, t ad,'1;le wha m n omQ31)﻿&C whe thoupl,"
werfrd s hed Disarm beron﻿812Kbirs r lkep Zerthed winte bbes y fo
raclsalerat  w im h ol y gsio t wed itop, ceanithersen'" usit  "

CUQa agin renge."
Hy
yondiroreoo;je tocaharer she waged the, afuf wind a, s?Lerun'UDvibedsst!n 'beit or.
cin ero
